In [1]:
!pip install python-twitter

  Using cached python_twitter-3.5-py2.py3-none-any.whl (67 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)


In [20]:
import twitter
import math
import urllib.request
import os
import config as cf

In [21]:
api = twitter.Api(consumer_key = cf.credentials["consumer_key"],
                  consumer_secret = cf.credentials["consumer_secret"],
                  access_token_key = cf.credentials["access_token"],
                  access_token_secret = cf.credentials["access_token_secret"])

In [26]:
hashtag = 'ميمز'
result_type = 'mixed' # possible values: mixed, recent, popular
include_entities = 'true'
with_twitter_user_id = 'true' # include user information
since = '2020-03-10' # start date
until = '2021-04-22'
count = '20' # The number of tweets to return per page

In [27]:
query = ('q={hashtag}' + 
         '&result_type={result_type}' +
         '&include_entities={include_entities}' +
         '&with_twitter_user_id={with_twitter_user_id}' + 
         '&since={since}' + 
         '&until={until}' +
         '&count={count}')

query = query.format(hashtag=hashtag,
                 result_type=result_type,
                 include_entities=include_entities,
                 with_twitter_user_id=with_twitter_user_id,
                 since=since,
                 until=until,
                 count=count)

In [28]:
query

'q=ميمز&result_type=mixed&include_entities=true&with_twitter_user_id=true&since=2020-03-10&until=2021-04-22&count=20'

In [29]:
all_results = []
max_id = None
IDs = []

for i in range(0,100):
    
    results = api.GetSearch(raw_query = query)
    all_results.extend(results)
    IDs = [result.id for result in results]
    print(IDs)
    smallest_ID = min(IDs)
    
    if max_id == None: # first call 
        max_id = smallest_ID
        query += '&max_id={max_id}'.format(max_id=max_id)
    else:
        old_max_id = "max_id={max_id}".format(max_id=max_id)
        max_id = smallest_ID
        new_max_id = "max_id={max_id}".format(max_id=max_id)
        query = query.replace(old_max_id,new_max_id)

[]


ValueError: min() arg is an empty sequence

In [8]:
print("last max_id =",max_id)

last max_id = 1383452693776138244


In [9]:
len(all_results)

1976

In [10]:
all_results[0]

Status(ID=1384999839927345158, ScreenName=p0urIavie, Created=Wed Apr 21 22:37:48 +0000 2021, Text='ميمز نهاية العالم كانوا كل شي https://t.co/foxETd8d5g')

In [11]:
all_results[0].id_str

'1384999839927345158'

In [12]:
all_results[0].user.screen_name

'p0urIavie'

In [13]:
sample_result = all_results[1]

tweet_id = sample_result.id_str
screen_name = sample_result.user.screen_name

tweet_url = "https://twitter.com/{screen_name}/status/{tweet_id}"
tweet_url = tweet_url.format(screen_name=screen_name,tweet_id=tweet_id)

print(tweet_url)

https://twitter.com/xaiuud/status/1384999347230740480


In [14]:
def get_tweet_url(tweet):
    
    tweet_id = tweet.id_str
    screen_name = tweet.user.screen_name

    tweet_url = "https://twitter.com/{screen_name}/status/{tweet_id}"
    tweet_url = tweet_url.format(screen_name=screen_name,tweet_id=tweet_id)
    
    return tweet_url

In [15]:
folder_name = "memes_pics"

In [16]:
image_origins = {
    "tweet_url": [],
    "image_id": [],
    "image_url": [],
}

downloaded_img_ids = [file[:file.find('.')] for file in os.listdir(folder_name)]

for tweet in all_results:
    
    tweet_url = get_tweet_url(tweet)
    
    if tweet.media:
        
        for media in tweet.media:
            
            media_id = str(media.id)
            media_url = media.media_url
            
            if not(media_id in downloaded_img_ids): # don't re-download images
                
                file_name = media_id
                file_type = os.path.splitext(media_url)[1]

                urllib.request.urlretrieve(media_url, os.path.join(folder_name,file_name+file_type))
                
                downloaded_img_ids.append(media_id)
                
                # save image origin info
                image_origins["tweet_url"].append(tweet_url)
                image_origins["image_id"].append(media_id)
                image_origins["image_url"].append(media_url)

In [17]:
import pickle

pickle.dump(image_origins,open("image_origins.p","wb"))
pickle.dump(all_results,open("all_results.p","wb"))

In [18]:
import pandas as pd

# turn dictionary to dataframe
image_info_df = pd.DataFrame.from_dict(image_origins)
# remove rows with duplicate image IDs
image_info_df = image_info_df.drop_duplicates(subset='image_id', keep='first')
# order columns
image_info_df = image_info_df[['image_id','image_url','tweet_url']]

In [20]:
image_info_df

,image_id,image_url,tweet_url
0,1384999835372187649,http://pbs.twimg.com/media/EziCAyFVcAElMjb.jpg,https://twitter.com/p0urIavie/status/138499983...
1,1384985398443757573,http://pbs.twimg.com/media/Ezh04cVX0AUszen.jpg,https://twitter.com/IBRA7IM_21/status/13849940...
2,1384989082821005316,http://pbs.twimg.com/ext_tw_video_thumb/138498...,https://twitter.com/Mkolp20709913/status/13849...
3,1384987416629555203,http://pbs.twimg.com/media/Ezh2t6qXMAMyRE6.jpg,https://twitter.com/xravzr/status/138498741950...
4,1384985630116098051,http://pbs.twimg.com/media/Ezh1F7YXMAMk8Of.jpg,https://twitter.com/AlTaie__/status/1384985633...
...,...,...,...
364,1383409639073587203,http://pbs.twimg.com/ext_tw_video_thumb/138340...,https://twitter.com/alex11496592/status/138346...
365,1383463649570824199,http://pbs.twimg.com/media/EzMM3ABVoAc2tMo.jpg,https://twitter.com/MadeInChina3005/status/138...
366,1383461365898104837,http://pbs.twimg.com/media/EzMKyErVUAUu0vf.jpg,https://twitter.com/SaiyukiG/status/1383461370...
367,1383460495382827011,http://pbs.twimg.com/ext_tw_video_thumb/138346...,https://twitter.com/GTX20192771/status/1383460...


In [21]:
len(image_info_df)

369

In [22]:
image_info_df.to_csv("image_origins.csv",index=False)

The `Status` class represents the Status structure used by the twitter API. It has the following attributes (all are None by default):
* contributors
* coordinates
* created_at: Date the tweet was published. e.g. 'Mon Jul 22 17:17:39 +0000 2019'
* current_user_retweet
* favorite_count: # of times the tweet has been liked (heart button).
* favorited: Boolean
* full_text: List of hashtags the post has. Each element is an object of the class `Hashtag`.
* geo
* hashtags
* id: ID which identifies tweet e.g. 1153353426291572736
* id_str: id as a string. e.g. '1153353426291572736'
* in_reply_to_screen_name
* in_reply_to_status_id
* in_reply_to_user_id
* lang: language of post. e.g. 'en'
* location
* media: List of objects of class `Media`, containing information of media present in the tweet, with URL to access the media. e.g. `[Media(ID=1153108337967468544, Type=video, DisplayURL='pic.twitter.com/IsRTsXfDs9')]`
* place
* possibly_sensitive: Boolean. e.g. False
* quoted_status: Twitter status shared by poster (retweet), if one was shared. It is an object of the class `Status`.
* quoted_status_id: id which identifies the shared post
* quoted_status_id_str: id of shared post as a string
* retweet_count: Number of times post was retweeted
* retweeted: Boolean
* retweeted_status
* scopes
* source: Device/source where post was shared. e.g. `<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>`
* text: Text of post, and link of retweet post of the post has a shared tweet. e.g. 'The people have spoken. #RickyRenuncia https://t.co/7UmyDL3Ry4'
* truncated: Boolean
* urls: list of urls mentioned in post, will usually be retweeted post. Each element is an object of the class `URL`. e.g. `[URL(URL=https://t.co/7UmyDL3Ry4, ExpandedURL=https://twitter.com/davidbegnaud/status/1153344412107399169)]`
* user: object of class `User`, has info of tweet author (user ID, screen name, etc)
* user_mentions: list
* withheld_copyright 
* withheld_in_countries
* withheld_scope

The `Hashtag` class represents a twitter hashtag. It has the following attributes:
* text: hashtag text (e.g. RickyRenuncia)

The `URL` class represents a URL contained in a tweet. It has the following attributes:
* expanded_url
* url

The `User` class represents the User structure. It has the following attributes:
* contributors_enabled
* created_at
* default_profile
* default_profile_image
* description
* email
* favourites_count
* followers_count
* following
* friends_count
* geo_enabled
* id
* id_str
* lang
* listed_count
* location
* name
* notifications
* profile attributes ...
* protected
* screen_name
* status
* statuses_count
* time_zone
* url
* utc_offset
* verified
* withheld_in_countries
* withheld_scope

The `Media` class represents the media component of a tweet. It has the following attributes:
* display_url
* expanded_url
* ext_alt_text
* id
* media_url
* media_url_https
* sizes
* type
* url 
* video_info